In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
BASE_URL = "https://fastwork.co/"

In [3]:
main_cat_urls = [
    'design-graphic',
    'marketing-advertising',
    'writing-translation',
    'photography-video',
    'web-programming',
    'consultant',
    'operation'
]

In [4]:
def retrieve_products(subcat_slug):
    url = "https://f24luz3xrm-dsn.algolia.net/1/indexes/th_prod_products/query"
    params = {
        "x-algolia-agent":"Algolia for JavaScript (3.35.1); Browser",
        "x-algolia-application-id":"F24LUZ3XRM",
        "x-algolia-api-key":"a639c2213baa3a66e57cee665d7e17a6"
    }
    data = {
        "attributesToRetrieve": ["*"],
        "facets": ["subcategory.title","subcategory.slug","base_price","user.first_name","tags"],
        "filters":"(subcategory.slug: {})".format(subcat_slug),
        "hitsPerPage":"100",
        "page":"0",
        "query":"",
    }
    response = requests.post(url,params=params,json=data)
    return response

In [ ]:
df_lists = []
for main_cat_url in main_cat_urls:
    response = requests.get(BASE_URL+main_cat_url)
    soup = BeautifulSoup(response.content, "html.parser")
    dom = soup.find_all('div',attrs={"class":"category-list"})
    sub_cat_links = dom[0].find_all('a')
    print("#{}".format(main_cat_url))
    
    for link in sub_cat_links:
        sub_cat_slug = link.attrs['href'].replace("/","")
        
        
        response = retrieve_products(sub_cat_slug)
        data = response.json()['hits']
        
        print("##{} - Count : {}".format(sub_cat_slug,len(data)))
        df = pd.DataFrame(data)
        df['category'] = main_cat_url
        df['subcategory']= sub_cat_slug
        df_lists.append(df)
        
df_all = pd.concat(df_lists)

#design-graphic
##logo-design - Count : 100
##illustration - Count : 100
##social-media-banner - Count : 100
##character-design - Count : 100
##packaging - Count : 100
##fanart - Count : 100
##illustration-portrait - Count : 100
##engineering-structural-design - Count : 100
##caricature - Count : 34
##architect-and-interior - Count : 100
##cartoon-drawing - Count : 100
##presentation - Count : 100
##vector-illustration - Count : 64
##publication - Count : 100
##vector - Count : 78
##storyboard - Count : 16
##fashion-design - Count : 100
##tattoo-design - Count : 27
##infographics - Count : 100
##map-design - Count : 55
##photoshop - Count : 100
##game-character-bg - Count : 23
##sticker - Count : 41
##perspective - Count : 100
##3d-model - Count : 100
##line-sticker - Count : 100
##portfolio-resume - Count : 100
##corporate-identity - Count : 30
##landscape - Count : 37
##nft-art - Count : 5
##design-graphic-other - Count : 31
#marketing-advertising


In [ ]:
labels = ["{0}".format(i) for i in range(0, 200000, 500)]

In [ ]:
df_all['bin_price'] = pd.cut(df_all['base_price'],bins=range(0,200250,500),right=False,labels=labels)

In [ ]:
print(df_all.head(30))

In [ ]:
plt.figure(figsize=(16, 9))
plt.xticks(rotation=45)
# ax = sns.scatterplot(df_all,x="category",y="bin_price")
ax = sns.swarmplot(df_all,x="category",y="bin_price")